# Segmenting and Clustering Neighborhoods in London


## Applied Data Science Capstone - complete report
by R. Shen on 9 Sept., 2019; special thanks to [Arvind Saigal](https://github.com/saigalarvind) & [Jason L Urquhart](https://github.com/JasonLUrquhart)

## Introduction to the opportunity

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://images.unsplash.com/photo-1505761671935-60b3a7427bad?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80")

People of all kinds from around the world flock to London, UK with some of these them aspiring to make this remarkable place a home of their own. With varying budgets and needs, people find it very hard to find a suitable place and neighborhood to accommodate them and their families. Due to high cost of living and other multiple issues, London housing has been struggling. With the inevitable Brexit, the problem has further compounded. A potential client aspiring to buy a suitable property would like to become knowledgeable about the ongoing pricing to make a concious decision. Further, he/ she would like to consider several factors like proximity to schools, medical care, restaurants to accommodate his/ her familial needs.

With government provided authentic data on London properties coupled with data science techniques, one can make derive the useful information about current pricing in different localities of London while considering other factors of his choice. This would help the potential client to make an informed decision about buying a suitable property. 


There will be a group of potential investors and clients seeking to purchase an appropriate property in London however are highly skeptical due to a lack of knowledge of the volatile market conditions as well as the potential impact of Brexit, especially in a no-deal scenario.

The stakeholders include HM Government, Property sellers, Clients / investors and real estate agents.

## Datasets used

- HM Land Registry: Price paid data 
    - _Open Data published by Government of UK_
    - Price Paid Data includes information on all property sales in England and Wales that are sold for full market value and are lodged with us for registration.
    - The dataset includes the transactions received at HM Land Registry in the lastest period (June 2019)
    - The dataset was downloaded ahead of time for convenience.

- Google Maps Geocoding API  
    - _Google Cloud Platform/ Google Maps_
    - Location coordinates obtained by Gmaps API calls.
    - Location Information obtained from Price Paid Dataset is used to obtain the location coordinates from Google Maps.
    - A separate Python script has been developed to extract the unique street names, district names from the Price Paid Dataset and embed those in the GMaps API calls to obtain the required information.
- Foursquare location data  
    - _Foursquare location data_
    - Location coordinates obtained by Foursquare API calls.
    - To determine the proximity of various amenities as per the client’s requirement, Foursquare location data is used.


## Methodology
1. **Loading each data set**
2. **Examine the price paid by neighbourhood**
3. **Study the property types and then pivot analysis by neighbourhood**
4. **Understand correlation between various parameters**
5. **Perform k-means statisical analysis on venues by locations of interest based on findings from prices and neighbourhood**
6. **Determine which venues are most common statistically in the region of greatest investment value then in all other locations of interest.**
7. **Determine if an area, would worth investment**

In [2]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files
import requests

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ebbc1554a11144158122a412ce1905fc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IQKOTWUJUElGha52i5hkNHbC0h_6bjvRLuiCYII-S6Uq',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ebbc1554a11144158122a412ce1905fc.get_object(Bucket='courseracapstone-donotdelete-pr-0akotni5ciarwz',Key='ppd_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_ppd = pd.read_csv(body)
df_ppd.head()

,unique_id,price_paid,deed_date,postcode,property_type,new_build,estate_type,saon,paon,street,...,L,FLAT 6,"BILL FAUST HOUSE, 20",TARLING STREET,Unnamed: 25,LONDON,TOWER HAMLETS,GREATER LONDON,A,http://landregistry.data.gov.uk/data/ppi/transaction/8A78B2AF-785C-5CB0-E053-6B04A8C0F504/current
0,85866A65-8ED3-143F-E053-6B04A8C06A15,400000,2019-03-06,E1 0AS,F,N,L,FLAT 19,VOGLER HOUSE,CABLE STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8A78B2AF-77E3-5CB0-E053-6B04A8C0F504,400000,2019-05-21,E1 0BJ,F,N,L,FLAT 11,BROCKMER HOUSE,CROWDER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8CAC1318-268A-0253-E053-6B04A8C08E51,432500,2019-06-03,E1 0DF,F,N,L,NaN,88,SOLANDER GARDENS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8A78B2AF-77F2-5CB0-E053-6B04A8C0F504,415000,2019-03-29,E1 0EA,F,N,L,NaN,32,DEVONPORT STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87E1551D-DB6D-6405-E053-6C04A8C0B2EE,160000,2019-03-28,E1 0HR,F,N,L,FLAT 4,"ROSEA HOUSE, 4",BOULCOTT STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Check column names
df_ppd.columns

Index(['unique_id', 'price_paid', 'deed_date', 'postcode', 'property_type',
       'new_build', 'estate_type', 'saon', 'paon', 'street', 'locality',
       'town', 'district', 'county', 'transaction_category',
       'linked_data_uri"8A78B2AF-785C-5CB0-E053-6B04A8C0F504"', '140000',
       '2019-05-24', 'E1 0AD', 'F', 'N', 'L', 'FLAT 6', 'BILL FAUST HOUSE, 20',
       'TARLING STREET', 'Unnamed: 25', 'LONDON', 'TOWER HAMLETS',
       'GREATER LONDON', 'A',
       'http://landregistry.data.gov.uk/data/ppi/transaction/8A78B2AF-785C-5CB0-E053-6B04A8C0F504/current'],
      dtype='object')

In [5]:
# Assign meaningful column names
df_ppd.drop(columns=['linked_data_uri"8A78B2AF-785C-5CB0-E053-6B04A8C0F504"', '140000',
       '2019-05-24', 'E1 0AD', 'F', 'N', 'L', 'FLAT 6', 'BILL FAUST HOUSE, 20',
       'TARLING STREET', 'Unnamed: 25', 'LONDON', 'TOWER HAMLETS',
       'GREATER LONDON', 'A',
       'http://landregistry.data.gov.uk/data/ppi/transaction/8A78B2AF-785C-5CB0-E053-6B04A8C0F504/current'], inplace=True)
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration','SAON','PAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

**Calcuate the street-wise average price of various properties**

In [7]:
df_grp_price = df_ppd.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

### With the buget limit given for purchasing a house...

**Find the locations df_grp_price which fits your budget**


_Please change the limits as per your budget_

In [8]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [9]:
# Display the dataframe
df_affordable

,Street,Avg_Price
136,ALBION SQUARE,2.450000e+06
232,AMBROSDEN AVENUE,2.210000e+06
371,ASHCHURCH GROVE,2.425000e+06
499,BALCOMBE STREET,2.217500e+06
557,BARHAM ROAD,2.225000e+06
621,BATHGATE ROAD,2.300000e+06
658,BEAUFORT GARDENS,2.325000e+06
704,BEECHWOOD AVENUE,2.500000e+06
821,BETTERTON STREET,2.500000e+06
962,BOLSOVER STREET,2.422962e+06


## Read the street-wise coordinates into a dataframe

In [10]:
body = client_ebbc1554a11144158122a412ce1905fc.get_object(Bucket='courseracapstone-donotdelete-pr-0akotni5ciarwz',Key='Street Coordinates London.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

london_data = pd.read_csv(body)
london_data.head()

,ID,Street,Latitude,Longitude,Unnamed: 4
0,136,ALBION SQUARE,51.540723,-0.07212,NaN
1,232,AMBROSDEN AVENUE,51.496063,-0.138763,NaN
2,371,ASHCHURCH GROVE,51.500699,-0.241592,NaN
3,499,BALCOMBE STREET,51.523221,-0.161897,NaN
4,557,BARHAM ROAD,51.423735,0.063186148477775,NaN


In [11]:
# Eliminate recurring word LONDON from individual names
london_data['Street'] = london_data['Street'].str.replace(', LONDON', '')

**Join the data to find the coordinates of locations which fit into client's budget**

In [12]:
df_pref_loc = pd.merge(london_data, df_affordable, on=['Street'], how='inner')
df_pref_loc.drop(columns=['Unnamed: 4'], inplace=True)
df_pref_loc.head()

,ID,Street,Latitude,Longitude,Avg_Price
0,136,ALBION SQUARE,51.540723,-0.07212,2450000.0
1,232,AMBROSDEN AVENUE,51.496063,-0.138763,2210000.0
2,371,ASHCHURCH GROVE,51.500699,-0.241592,2425000.0
3,499,BALCOMBE STREET,51.523221,-0.161897,2217500.0
4,557,BARHAM ROAD,51.423735,0.063186148477775,2225000.0


## Plot recommended locations on map of London with current market prices

In [13]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.4893335, -0.144055084527687.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [16]:
# create map of london using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map

'''for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Avg_Price'], df_pref_loc['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  '''

map_london

In [ ]:
CLIENT_ID = '4IFTJO5GDDJSGFD11ZSRL3EYNPXKLTNH0VB3INRNVARVWBR4' # your Foursquare ID
CLIENT_SECRET = 'TRFF5K5BL3TABXJRZKRSHN3X0BV4UGJE35DTIDDYZYU34AKE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Let's explore the first neighborhood in our dataframe.

## Explore the preferred location for different venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each location and create a new dataframe called location_venues and display it.

In [ ]:
# type your answer here
location_venues = getNearbyVenues(names=df_pref_loc['Street'],
                                   latitudes=df_pref_loc['Latitude'],
                                   longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_venues

In [ ]:
location_venues.groupby('Street').count()

In [ ]:
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [ ]:
location_facility = getNearbyFacility(names=df_pref_loc['Street'],
                                      latitudes=df_pref_loc['Latitude'],
                                      longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_facility

In [ ]:
location_facility.groupby('Street Name').count()

## Discussion
Based upon the findings in the results section, the user can take a conscious decision about choosing a street/ location based upon his/ her requirements.
The results section enlists 39 places where a prospective client can buy a property based upon his needs and choices. Such choices would be affected by the venues and facilities which are close to the property which match against his familial needs.
Few possible cases are: -
1. A prospective client with elders in the family would be inclined to choose a location where hospitals and grocery stores are located in close proximity.
2. A prospective client with kids in the family would choose a location where elementary and high schools are close-by. He would also like to choose a place with parks and other venues to accommodate his family are in the close vicinity.
3. A bachelor would be inclined to choose a property which has pubs, bars, entertainment places close to the property.

## Conclusion
The decision of a buyer is influenced by the familial needs, personal biases. So, based upon the findings summarized in the results and discussion sections, following conclusions can be made: -
1. While making recommendations to a prospective client, it is imperative to know his/ her immediate needs and requirements besides the budget. This would help to catch his/ her attention.
2. Knowledge about the most recent market prices can be very helpful for the client and can help him take a decision.